In [1]:
#Heap stuff 
type Node
    # a node data structure. It contains two arguments: a key and a value. "key" 
    # is the used to rank the node, possibly by a heap. "val" is the value of the 
    # node from the perspective of its graph.
    key::Real
    val::Real
end

type Heap
    # a heap data structure. It contains a list of node objects. Note that elements 
    # are never deleted from the node list; rather, a pointer called "last" is used 
    # to indicate that last good value in the arrays. The heaps can be either min or 
    # max heaps, as determined by the keyword argument "kind". An address book is 
    # maintained that tells where a node of particluar value (not key) is located 
    # within the heap.
    list::Array{Node,1}
    last::Int
    kind::String
    addressbook::Dict{Real,Real}
end
Heap(kind) = Heap([], 0, kind, Dict())
Heap(node::Node, kind) = Heap([node], 1, kind, Dict(node.val => node.key))
Heap(nodelist::Array{Node,1}, kind) = heapify(Heap(nodelist, size(nodelist)[1], kind, Dict()))

function heapify(h)
    # arguments:
    #     h: a Heap data structure, but without the true min or max heap property
    # what it does:
    #     bestows the true min or max heap property upon h. It does this by sinking 
    #     each parent node in turn from bottom to top. Once all parent nodes have 
    #     been sunk to their proper positions in the heap, true heap property is 
    #     established.
    # returns:
    #    a proper min or max heap
    for n = h.last>>1:-1:1
        sink(h, n)
    end
    for i = 1:h.last
        h.addressbook[h.list[i].val] = i
    end
    return h
end
    
function sink(heap, i)
    # arguments:
    #     heap: a heap data structure
    #     i: an index describing the location in the heap (in the base array 
    #     itself) at which the node to be sunk can be found 
    # what it does:
    #     after the base node is popped from the heap, the node in the last 
    #     position is brought into the base node, at least temporarily. The proper 
    #     position for this node can be found by "sinking" it down through the heap.
    #     The address book is updates every time a node is sunk
    # returns:
    #    return statements are for facilitation of recursion only
    if heap.last != 0
        if heap.kind == "min"
            ∘ = <=
        elseif heap.kind == "max"
            ∘ = >=
        end
        l = i<<1
        r = i<<1+1
        if (r <= heap.last && 
                heap.list[r].key ∘ heap.list[l].key && 
                heap.list[r].key ∘ heap.list[i].key)
            heap.list[i], heap.list[r] = heap.list[r], heap.list[i]
            heap.addressbook[heap.list[i].val] = i
            heap.addressbook[heap.list[r].val] = r
            return sink(heap, r)
        elseif l <= heap.last && heap.list[l].key ∘ heap.list[i].key
            heap.list[i], heap.list[l] = heap.list[l], heap.list[i]
            heap.addressbook[heap.list[i].val] = i
            heap.addressbook[heap.list[l].val] = l
            return sink(heap, l)
        else
            return
        end
    end
end

function float(heap, i)
    # arguments:
    #     heap: a heap data structure
    #     i: an index describing the location in the heap (in the base array 
    #     itself) of the node to be floated can be found 
    # what it does:
    #     When a new node is pushed onto the heap in the last position, or when the 
    #     key of a node is decreased/increased (min/max), we need to check if this 
    #     new node must be moved upward to the top of the heap. The proper position 
    #     for this node can be found by "floating" it up through the heap. The
    #     address book is updated every time a node is floated
    # returns:
    #    return statements are for facilitation of recursion only
    if i == 1
        return
    else
        if heap.kind == "min"
            ∘ = <
        elseif heap.kind == "max"
            ∘ = >
        end
        if heap.list[i].key ∘ heap.list[i>>1].key
            heap.list[i], heap.list[i>>1] = heap.list[i>>1], heap.list[i]
            heap.addressbook[heap.list[i].val] = i
            heap.addressbook[heap.list[i>>1].val] = i>>1
            return float(heap, i>>1)
        end
    end
end
        
function pop(heap)
    # arguments:
    #     heap: a heap data structure
    # what it does:
    #     removes the base node from the heap. Note that elements are never 
    #     deleted from the heap's lists, rather the last pointer is decremented.
    #     The address book is updated every time a node is popped.
    # returns:
    #    the contents of the base node: its key and value. If the heap is empty 
    #    it returns an infinity-stuffed node
    returnable = heap.list[1]
    heap.list[1], heap.list[heap.last] = heap.list[heap.last], heap.list[1]
    heap.addressbook[heap.list[1].val] = 1
    heap.addressbook[heap.list[heap.last].val] = heap.last
    heap.last -= 1
    sink(heap, 1)
    return returnable
end

function push(heap, nuno)
    # arguments:
    #     heap: a heap data structure
    #     nuno: the node being pushed onto the heap
    # what it does:
    #     adds a new node to the heap. Note that if the heap's lists are too short to 
    #     contain the new nodes children, dummy elements are added to the lists. The 
    #     keys of these dummies are the maximum allowed by the data type if a min heap, 
    #     zero if a max heap. This is done to facilitate sinking after pop.
    # returns:
    #    no returns
    heap.last += 1
    if size(heap.list)[1] < heap.last
        push!(heap.list, nuno)
    else
        heap.list[heap.last] = nuno
    end
    float(heap, heap.last)
end

function rekey(heap, val, key)
    # arguments:
    #     heap: a heap data structure
    #     val: the unique value of some node in the heap
    #     key: the key by which the node of value val is ordered in the heap
    # what it does:
    #     decreases (min heap) or increases (max heap) the key associated with some node 
    #     if a smaller or larger key is discovered during graph search
    # returns:
    #    no returns
    if heap.addressbook[val] <= heap.last
        if heap.kind == "min"
            ∘ = <
        elseif heap.kind == "max"
            ∘ = >
        end
        if key ∘ heap.list[heap.addressbook[val]].key
            heap.list[heap.addressbook[val]].key = key
            float(heap, heap.addressbook[val])
        end
    end
end

rekey (generic function with 1 method)

In [2]:
function Dijkstra(graph, startnode)
    # arguments:
    #     graph: an undirected adjacency list of the form Dict{Any,Dict{Any,Number}}.
    #     startnode: the starting point from which all shortest paths will be measured
    # what it does:
    #     finds the shortest path to all points from the startnode
    # returns:
    #    a complete list of nodes sorted in order of node number. Each node in the 
    #    list contains the shortest path as its "key", and the node number as its "val"
    n = size(collect(keys(graph)))[1]
    shortestpaths = []
    preheaped = [k==startnode ? Node(0,k) : Node(Inf, k) for k=1:n]
    heap = Heap(preheaped, "min")
    while heap.last > 0
        current = pop(heap)
        push!(shortestpaths, current)
        for oppnode in keys(graph[current.val])
            rekey(heap, oppnode, current.key + graph[current.val][oppnode])
        end
    end
    return shortestpaths
end

Dijkstra (generic function with 1 method)

# An Example


This data which is loaded in the cell below is the proper form of the graph to feed to the Dijkstra algorithm. The structure must be of the form Dict{Int, Dict{Int, Real}}. Interpret its meaning through the following example, that of node 1. There are a number of undirected edges leading out of it, including edges to node 169 with edge length 2001 and node 11 with edge length 70.

In [3]:
using JLD
adj = load("dijkstra.jld", "al") 

adj[1]

INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/JLD.ji for module JLD.


Dict{Int64,Real} with 15 entries:
  169 => 2001
  11  => 70
  91  => 2446
  26  => 2838
  10  => 1918
  138 => 7783
  156 => 5786
  177 => 3779
  61  => 8958
  78  => 6878
  154 => 2697
  67  => 11
  36  => 8196
  164 => 7749
  33  => 5564

Here are the results of the test run. The nodes are printed in order of shortest path length fron Node 1.

In [4]:
@time Dijkstra(adj, 1)

  0.971730 seconds (253.49 k allocations: 10.686 MB)


200-element Array{Any,1}:
 Node(0,1)     
 Node(11,67)   
 Node(70,11)   
 Node(72,72)   
 Node(168,94)  
 Node(450,73)  
 Node(594,196) 
 Node(724,130) 
 Node(770,133) 
 Node(778,136) 
 Node(815,3)   
 Node(864,74)  
 Node(897,160) 
 ⋮             
 Node(3807,76) 
 Node(3860,40) 
 Node(3861,90) 
 Node(3997,184)
 Node(4102,31) 
 Node(4127,45) 
 Node(4226,173)
 Node(4348,141)
 Node(4357,95) 
 Node(4629,168)
 Node(4950,70) 
 Node(5403,163)